# Trabalho prático de Elementos de inteligencia artificial


In [ ]:
import pandas as pd
import requests
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
from distritos import dicionario_distritos

## Recolha de Dados

In [358]:
LINKS = [
    "https://www.pordata.pt/pt/estatisticas/saude/pessoal-de-saude/pessoal-ao-servico-nos-hospitais-portugal",
    "https://www.pordata.pt/pt/estatisticas/populacao/esperanca-de-vida-e-obitos/taxa-bruta-de-mortalidade-0",
    "https://www.pordata.pt/pt/estatisticas/populacao/nascimentos-e-fecundidade/taxa-bruta-de-natalidade-0"
]

dataframes: dict[str, pd.DataFrame] = {
    "pessoal-servico": None,
    "mortalidade": None,
    "natalidade": None
}

c = -1
for url in LINKS:
    c += 1
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Erro ao tentar acessar ao site: {url}")
        continue
    
    soup = BeautifulSoup(response.text, "html.parser")

    download_btn = soup.find("a", id="download-btn")

    if not download_btn or not download_btn.has_attr("href"):
        print(f"Erro ao tentar acessar o csv do link {url}")
        continue
    
    match c:
        case 0:
            dataframes["pessoal-servico"] = pd.read_csv(download_btn["href"])
        case 1:
            dataframes["mortalidade"] = pd.read_csv(download_btn["href"])
        case 2:
            dataframes["natalidade"] = pd.read_csv(download_btn["href"])

## Integração de Dados

#### Pessoal ao serviço nos hospitais

In [359]:
# Deixar só os municipios e apagar dados gerais nacionais
dataframes["pessoal-servico"] = dataframes["pessoal-servico"].dropna(subset=["03. Âmbito Geográfico"])

#Apagar colunas futeis
dataframes["pessoal-servico"] = dataframes["pessoal-servico"].drop(columns=["03. Âmbito Geográfico", "06. Filtro 2", "07. Filtro 3", "08. Escala", "09. Símbolo"])

#Mudar nome das colunas
dataframes["pessoal-servico"] = dataframes["pessoal-servico"].rename(columns={"01. Ano": "Ano",
                                                                              "02. Nome Região (Portugal)": "Nome Municipio",
                                                                              "04. Indicador": "Tipo de Hospital",
                                                                              "05. Filtro 1": "Tipo de Pessoal Medico",
                                                                              "10. Valor": "Quantidade de Pessoal Medico"
                                                                              })

#Mudar o tipo da coluna do ano para um numero inteiro
dataframes["pessoal-servico"][dataframes["pessoal-servico"].columns[0]] = dataframes["pessoal-servico"][dataframes["pessoal-servico"].columns[0]].astype(int)

#Separar tipo de pessoal medico
dataframes["pessoal-servico"] = dataframes["pessoal-servico"].pivot_table(
    index=["Ano", "Nome Municipio", "Tipo de Hospital"],
    columns="Tipo de Pessoal Medico",
    values="Quantidade de Pessoal Medico",
    aggfunc="sum"
).reset_index()

# Remover primeira coluna criada depois do pivot
dataframes["pessoal-servico"].columns.name = None

#Reordenar as colunas
dataframes["pessoal-servico"] = dataframes["pessoal-servico"][["Ano", "Nome Municipio", "Tipo de Hospital", "Enfermeiros",
                                                               "Médicos", "Pessoal auxiliar", "Técnicos de diagnóstico e terapêutica",
                                                               "Outros", "Total"
                                                              ]]

display(dataframes["pessoal-servico"].head(100))

,Ano,Nome Municipio,Tipo de Hospital,Enfermeiros,Médicos,Pessoal auxiliar,Técnicos de diagnóstico e terapêutica,Outros,Total
0,2002,Abrantes,Hospitais,207.0,73.0,159.0,34.0,132.0,605.0
1,2002,Aguiar da Beira,Hospitais,0.0,0.0,0.0,0.0,0.0,0.0
2,2002,Alandroal,Hospitais,0.0,0.0,0.0,0.0,0.0,0.0
3,2002,Albergaria-a-Velha,Hospitais,0.0,0.0,0.0,0.0,0.0,0.0
4,2002,Albufeira,Hospitais,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
95,2002,Ferreira do Zêzere,Hospitais,0.0,0.0,0.0,0.0,0.0,0.0
96,2002,Figueira da Foz,Hospitais,188.0,104.0,114.0,48.0,133.0,587.0
97,2002,Figueira de Castelo Rodrigo,Hospitais,0.0,0.0,0.0,0.0,0.0,0.0
98,2002,Figueiró dos Vinhos,Hospitais,0.0,0.0,0.0,0.0,0.0,0.0


#### Taxa Bruta de Mortalidade

In [360]:
# Deixar só os municipios e apagar dados gerais nacionais
dataframes["mortalidade"] = dataframes["mortalidade"].dropna(subset=["04. Âmbito Geográfico"])

#Apagar colunas futeis
dataframes["mortalidade"] = dataframes["mortalidade"].drop(columns=["02. Nome País (Europa)", "04. Âmbito Geográfico", "05. Filtro 1", "06. Filtro 2", "07. Filtro 3", "08. Escala", "09. Símbolo"])

#Mudar nome das colunas
dataframes["mortalidade"] = dataframes["mortalidade"].rename(columns={"01. Ano": "Ano",
                                                                      "03. Nome Região (Portugal)": "Nome Municipio",
                                                                      "10. Valor": "Taxa Bruta de Mortalidade (‰)"
                                                                    })

#Mudar o tipo da coluna do ano para um numero inteiro
dataframes["mortalidade"][dataframes["mortalidade"].columns[0]] = dataframes["mortalidade"][dataframes["mortalidade"].columns[0]].astype(int)

display(dataframes["mortalidade"].head(100))

,Ano,Nome Municipio,Taxa Bruta de Mortalidade (‰)
1792,1996,Abrantes,14.5
1793,1996,Águeda,8.7
1794,1996,Aguiar da Beira,17.0
1795,1996,Alandroal,15.5
1796,1996,Albergaria-a-Velha,9.0
...,...,...,...
1887,1996,Felgueiras,6.4
1888,1996,Ferreira do Alentejo,14.2
1889,1996,Ferreira do Zêzere,17.1
1890,1996,Figueira da Foz,12.2


#### Taxa Bruta de Natalidade

In [361]:
# Deixar só os municipios e apagar dados gerais nacionais
dataframes["natalidade"] = dataframes["natalidade"].dropna(subset=["04. Âmbito Geográfico"])

#Apagar colunas futeis
dataframes["natalidade"] = dataframes["natalidade"].drop(columns=["02. Nome País (Europa)", "04. Âmbito Geográfico", "05. Filtro 1", "06. Filtro 2", "07. Filtro 3", "08. Escala", "09. Símbolo"])

#Mudar nome das colunas
dataframes["natalidade"] = dataframes["natalidade"].rename(columns={"01. Ano": "Ano",
                                                                    "03. Nome Região (Portugal)": "Nome Municipio",
                                                                    "10. Valor": "Taxa Bruta de Natalidade (‰)",
                                                                    })


#Mudar o tipo da coluna do ano para um numero inteiro
dataframes["natalidade"][dataframes["natalidade"].columns[0]] = dataframes["natalidade"][dataframes["natalidade"].columns[0]].astype(int)

dataframes["natalidade"].head(100)

,Ano,Nome Municipio,Taxa Bruta de Natalidade (‰)
1792,1995,Abrantes,7.0
1793,1995,Águeda,10.6
1794,1995,Aguiar da Beira,9.0
1795,1995,Alandroal,6.3
1796,1995,Albergaria-a-Velha,10.7
...,...,...,...
1887,1995,Felgueiras,16.7
1888,1995,Ferreira do Alentejo,7.7
1889,1995,Ferreira do Zêzere,6.3
1890,1995,Figueira da Foz,9.1


#### Conjugação dos csv"s

In [362]:
#Remoção das colunas com os valores inferiores a 2002, pois o pessoal medico não tem valores sobre tais anos
dataframes["mortalidade"] = dataframes["mortalidade"][dataframes["mortalidade"]["Ano"] >= 2002]
dataframes["natalidade"] = dataframes["natalidade"][dataframes["natalidade"]["Ano"] >= 2002]

#Remoção das colunas com os valores de 2024 da mortalidade, pois tanto a natalidade quando o pessoal não tem desse ano
dataframes["mortalidade"] = dataframes["mortalidade"][dataframes["mortalidade"]["Ano"] < 2024]
display(dataframes["pessoal-servico"].head())


df_final = pd.merge(dataframes["pessoal-servico"], dataframes["mortalidade"])
df_final = pd.merge(df_final, dataframes["natalidade"])

,Ano,Nome Municipio,Tipo de Hospital,Enfermeiros,Médicos,Pessoal auxiliar,Técnicos de diagnóstico e terapêutica,Outros,Total
0,2002,Abrantes,Hospitais,207.0,73.0,159.0,34.0,132.0,605.0
1,2002,Aguiar da Beira,Hospitais,0.0,0.0,0.0,0.0,0.0,0.0
2,2002,Alandroal,Hospitais,0.0,0.0,0.0,0.0,0.0,0.0
3,2002,Albergaria-a-Velha,Hospitais,0.0,0.0,0.0,0.0,0.0,0.0
4,2002,Albufeira,Hospitais,0.0,0.0,0.0,0.0,0.0,0.0


#### Adição da coluna de distrito

In [ ]:
#Adição da coluna de distrito
df_final["Distrito"] = df_final["Nome Municipio"].map(dicionario_distritos)

#Reodenação para por a coluna de distrito em segundo
df_final = df_final[["Ano", "Distrito", "Nome Municipio", 
                     "Tipo de Hospital", "Enfermeiros", "Médicos",
                     "Pessoal auxiliar", "Técnicos de diagnóstico e terapêutica",
                     "Outros","Total", "Taxa Bruta de Mortalidade (‰)",
                     "Taxa Bruta de Natalidade (‰)"
                    ]]

df_final.to_csv("distritos.csv", index=False)

#### Restrição para apenas coluna de distrito

In [ ]:
# Remoção da coluna de municipio
df_final = df_final.drop(columns=["Nome Municipio"])

# Agrupação das colunas pelo ano e distrito
df_final = df_final.groupby(["Ano", "Distrito"], as_index=False).agg({
    "Enfermeiros": "sum",
    "Médicos": "sum",
    "Pessoal auxiliar": "sum",
    "Técnicos de diagnóstico e terapêutica": "sum",
    "Outros": "sum",
    "Total": "sum",
    "Taxa Bruta de Mortalidade (‰)": "mean",
    "Taxa Bruta de Natalidade (‰)": "mean"
})

df_final.head(100)

df_final.to_csv("final.csv", index=False)

,Ano,Distrito,Enfermeiros,Médicos,Pessoal auxiliar,Técnicos de diagnóstico e terapêutica,Outros,Total,Taxa Bruta de Mortalidade (‰),Taxa Bruta de Natalidade (‰)
0,2002,Distrito da Guarda,54.0,7.0,36.0,16.0,50.0,163.0,16.064286,7.250000
1,2002,Distrito de Aveiro,839.0,401.0,698.0,151.0,542.0,2631.0,9.852632,10.773684
2,2002,Distrito de Beja,315.0,117.0,196.0,55.0,246.0,929.0,17.335714,7.678571
3,2002,Distrito de Braga,686.0,468.0,647.0,125.0,507.0,2433.0,9.050000,11.778571
4,2002,Distrito de Bragança,408.0,107.0,261.0,66.0,226.0,1068.0,15.016667,6.833333
...,...,...,...,...,...,...,...,...,...,...
95,2006,Distrito de Viseu,192.0,62.0,88.0,31.0,147.0,520.0,12.004167,8.004167
96,2006,Distrito de Évora,0.0,0.0,0.0,0.0,0.0,0.0,13.435714,7.757143
97,2006,Distrito do Porto,4990.0,3481.0,4380.0,1025.0,4579.0,18455.0,7.344444,10.450000
98,2006,Região Autónoma da Madeira,1019.0,347.0,799.0,219.0,1246.0,3630.0,11.300000,10.481818


# Análise Exploratória

#### Verificar valores duplicados

In [ ]:
print("Linhas duplicadas:", int(df_final.duplicated().sum()))

Linhas duplicadas: 0


#### Verificar valores em falta

In [ ]:
print("Colunas com valores em falta:")
display(df_final.isnull().sum())

Colunas com valores em falta:


Ano                                      0
Distrito                                 0
Nome Municipio                           0
Tipo de Hospital                         0
Enfermeiros                              0
Médicos                                  0
Pessoal auxiliar                         0
Técnicos de diagnóstico e terapêutica    0
Outros                                   0
Total                                    0
Taxa Bruta de Mortalidade (‰)            0
Taxa Bruta de Natalidade (‰)             0
dtype: int64